### Basic Features

Implémentons avec les sommmes aggrégées de [2014](http://www2.impots.gouv.fr/documentation/statistiques/2042_nat/2015/revenus_2014_6e_ano.pdf)

In [2]:
import random
import pandas as pd
import numpy as np

In [48]:
situations = [['M', 12002841], ['D', 5510809], ['O', 983754], ['C', 14934477], ['V', 3997578]]
nbr_foyer = int(sum(w for c, w in situations))
print"Nombre de déclarations :",nbr_foyer

#to do : pour l'instant références implémentées en dur

Nombre de déclarations : 37429459


In [49]:
pourcent_situations = situations # connaitre la proportion pour distribuer
for x in pourcent_situations :
    x[1] = float(x[1])/nbr_foyer
pourcent_situations

[['M', 0.3206789871047829],
 ['D', 0.14723186354363282],
 ['O', 0.026282880551386008],
 ['C', 0.3990032824145281],
 ['V', 0.10680298638567018]]

In [7]:
openfisca_entry_variables = ['1AJ', '1AS', '1AP', '1AO'] # à compléter au fur et à mesure
index = np.arange(nbr_foyer/1000)
population = pd.DataFrame(columns=openfisca_entry_variables, index = index)
len(population)

37429

### Simulate 1AJ

In [ ]:
revenus = [21820704, 503723963299] # (nbr de déclarant en case 1aj, montant total de cette case) -> 2014
revenus_moy = revenus[1] / float(revenus[0])
pourcent= REVENUS[0] / float(nbr_foyer)

In [7]:
def generate_random_cerfa():
    cerfa = {}

    if random.random() < PERCENT_REVENUS_NOT_0 * 1.3:
        cerfa['1AJ'] = max(random.gauss(14000, 23500), 0)
    else:
        cerfa['1AJ'] = 0
    return cerfa

def gradiant(a, b):
    if a > b:
        return min((a / b - 1) * random.random(), 0.5)
    else:
        return min((b / a - 1) * random.random(), 0.5)


def find_gaussian_parameters(number_not_0, total_value, distribution_percentage_null=5):
    def simulate_population(th, mu, sigma, percentage_repr):
        total_result = 0
        number_not_null = 0
        for i in range(0, int(TOTAL_DECLARATIONS * percentage_repr)):
            result = simulate_one_gaussian(th, mu, sigma)
            total_result += result
            if result > 0:
                number_not_null += 1
        return number_not_null / percentage_repr, total_result / percentage_repr

    def simulate_one_gaussian(th, mu, sigma):
        if random.random() < th:
            return max(random.gauss(mu, sigma), 0)
        return 0

    # Between 0 and 1
    number_not_0 = float(number_not_0)
    total_value = float(total_value)

    percentage_repr = 0.001
    mu = total_value / number_not_0
    sigma = mu / 2
    mu_step = mu / 2
    sigma_step = sigma / 2
    th = (1 + distribution_percentage_null / 100.0) * number_not_0 / TOTAL_DECLARATIONS
    print repr(th)
    max_number_of_simulations = 100
    for i in range(0, max_number_of_simulations):
        sim_not_0, sim_tot_value = simulate_population(th, mu, sigma, percentage_repr)
        if sim_not_0 > number_not_0:
            mu -= mu_step * gradiant(number_not_0, sim_not_0)
            # sigma -= sigma_step * gradiant(number_not_0, sim_not_0)
        else:
            mu += mu_step * gradiant(sim_not_0, number_not_0)
            # sigma += sigma_step * gradiant(sim_not_0, number_not_0)

        if sim_tot_value > total_value:
            # mu -= mu_step * gradiant(total_value, sim_tot_value)
            sigma -= sigma_step * gradiant(total_value, sim_tot_value)
        else:
            # mu += mu_step * gradiant(sim_tot_value, total_value )
            sigma += sigma_step * gradiant(sim_tot_value, total_value)
        print 'Total target ' + str(sim_tot_value/total_value) + ' not 0 target: ' + str(sim_not_0/number_not_0) + ' mu=' +  repr(mu) + ' sigma=' + repr(sigma) + ' th=' + str(th)
        mu_step = mu_step * 0.995
        sigma_step = sigma_step * 0.995
        percentage_repr = percentage_repr * 1.01


find_gaussian_parameters(21820704, 503723963299, distribution_percentage_null=5)

0.6121311878966779
Total target 1.06453822891 not 0 target: 1.03351385913 mu=22723.014328775633 sigma=11362.332536653179 th=0.612131187897
Total target 1.03035048374 not 0 target: 1.02155682552 mu=22549.245620867325 sigma=11307.954516316233 th=0.612131187897
Total target 1.03434759828 not 0 target: 1.0291877932 mu=22285.563459143945 sigma=11142.39753860256 th=0.612131187897
Total target 1.02075438766 not 0 target: 1.03078507862 mu=22142.466064594988 sigma=11128.224484249376 th=0.612131187897
Total target 1.01235014023 not 0 target: 1.02599618626 mu=22080.190499220487 sigma=11065.082860432603 th=0.612131187897
Total target 1.01536529462 not 0 target: 1.02656434381 mu=21866.560491121967 sigma=11012.003560988995 th=0.612131187897
Total target 1.00561803319 not 0 target: 1.02879072811 mu=21832.577188361447 sigma=10982.971178238968 th=0.612131187897
Total target 1.00423170196 not 0 target: 1.02963279745 mu=21824.726688718758 sigma=10966.481547795713 th=0.612131187897
Total target 0.99278664

### Gestion des situations familiales
Idée : obtenir le nombre moyen d'enfant = 1.7

In [5]:
# Statistiques by familly, approximated to match the declaration d'impots
# Voir la distribution réelle mais bloquer par nbenf > 4 répartition en 3 plus calage car moyenne au-dessus de la réalité
CHILDREN_PER_FAMILY = [(1, 46), (2, 38.5), (3, 12.5), (4, 2), (5, 1)]
TOTAL_CHILDREN = float(sum(w*c for c, w in CHILDREN_PER_FAMILY))
print('TOTAL_CHILDREN = ', float(sum(w*c for c, w in CHILDREN_PER_FAMILY)))
# Familles avec enfants a charge de moins de 18 ans
FAMILLES = 9321480

('TOTAL_CHILDREN = ', 173.5)


In [ ]:
def weighted_choice(choices):
   total = sum(w for c, w in choices)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choices:
      if upto + w >= r:
         return c
      upto += w
   assert False, "Shouldn't get here"


    # Age is random between 18 and 88
    cerfa['0DA'] = int(random.random() * 70 + 18)

    # Drawing the situation
    # situation = weighted_choice(POSSIBLE_SITUATIONS)
    # cerfa[situation] = 1

    ## We only give children to married or pacces. This is an approximation
    # enfants = 0
    # if situation == 'M' or situation == 'O':
    #     if random.random() < (FAMILLES / float(POSSIBLE_SITUATIONS[0][1] + POSSIBLE_SITUATIONS[2][1])):
    #         enfants = weighted_choice(CHILDREN_PER_FAMILY)
    #
    # if enfants > 0:
    #     cerfa['F'] = enfants

    # Distribution that has a cool shape and required properties 5500, 26500